In [1]:
from glass.adv.txtcls.freq import correlated_words
from glass.pyt.df.stats import df_to_freqdf

In [2]:
DB = {'HOST' : 'localhost', 'PORT' : '5432', 'PASSWORD' : 'admin', 'USER' : 'postgres', 'DATABASE' : 'expvgi_api'}

Q = (
    "SELECT rd.*, reftxt.full_pp, reftxt.txt_pp "
    "FROM geotmlnh_refdata AS rd INNER JOIN geotmlnh_refdatatxt AS reftxt "
    "ON rd.fid = reftxt.rid"
)

REF_COL = 'is_fire'
DATA_COL = 'txt_pp'
OUT_FLD = r'D:\xpto\expvgi-inescc\api\geotmlnh\mdllog'

In [3]:
dt_ref = q_to_obj(DB, Q)
dt_ref.head()

,fid,is_fire,is_fire_bin,is_fire_evn,postid,full_pp,txt_pp
0,1,1,1,1,106349590991_10156962039030992,quas 150 operacion tres mei aer combat fog ser...,quase 150 operacionais três meios aéreos comba...
1,2,0,0,0,106349590991_10156965759890992,mulh mat filh rec nasc ter agid cium vinganc p...,mulher matou filha recém nascida agido ciúmes ...
2,3,0,0,0,106349590991_10156966055465992,atropel ex namor diz aperceb pod ter sid caixo...,atropelou ex namorada diz apercebeu podia ter ...
3,4,0,0,0,106349590991_10156966754535992,mulh mat filh rec nasc ter agid cium vinganc p...,mulher matou filha recém nascida agido ciúmes ...
4,5,0,0,0,106349590991_10156967976755992,hom morr desp mot pared alert dad 7 31 2018 10...,homem morre despiste moto paredes alerta dado ...


In [4]:
ref_Count = df_to_freqdf(dt_ref, REF_COL)

ref_Count['n_train'] = (ref_Count['count'] * TRAIN_DIM) / 100
ref_Count = ref_Count.round({'n_train' : 0})
ref_Count['n_train'] = ref_Count.n_train.astype(int)
ref_Count['count'] = ref_Count['count'].astype(int)

ref_Count.head()

,is_fire,count,percentage,n_train
0,0,1931,77.519069,1352
1,1,560,22.480931,392


In [5]:
dt_ref['RN'] = dt_ref.sort_values(
    [REF_COL], ascending=[True]
).groupby([REF_COL]).cumcount() + 1

dt_ref.head()

,fid,is_fire,is_fire_bin,is_fire_evn,postid,full_pp,txt_pp,RN
0,1,1,1,1,106349590991_10156962039030992,quas 150 operacion tres mei aer combat fog ser...,quase 150 operacionais três meios aéreos comba...,5
1,2,0,0,0,106349590991_10156965759890992,mulh mat filh rec nasc ter agid cium vinganc p...,mulher matou filha recém nascida agido ciúmes ...,1296
2,3,0,0,0,106349590991_10156966055465992,atropel ex namor diz aperceb pod ter sid caixo...,atropelou ex namorada diz apercebeu podia ter ...,1295
3,4,0,0,0,106349590991_10156966754535992,mulh mat filh rec nasc ter agid cium vinganc p...,mulher matou filha recém nascida agido ciúmes ...,1294
4,5,0,0,0,106349590991_10156967976755992,hom morr desp mot pared alert dad 7 31 2018 10...,homem morre despiste moto paredes alerta dado ...,1293


In [8]:
bestMdl = {}
for m in sintese:
    for i in sintese[m]:
        if not i:
            bestMdl[m] = i
        else:
            if sintese[m][i]["ACC"] > sintese[m][bestMdl[m]]["ACC"]:
                bestMdl[m] = i
            else:
                continue

print(bestMdl)

{'NB': 1, 'SVM': 1, 'RF': 0, 'LOGREG': 1}


In [10]:
print(sintese['NB'][1]["ACC"])
print(sintese['SVM'][1]["ACC"])
print(sintese['RF'][0]["ACC"])
print(sintese['LOGREG'][1]["ACC"])

91.834
92.771
93.708
92.771


In [ ]:
if TRAIN_DIM == 100:
    dt_train = dt_ref
    dt_cls   = dt_ref.copy()

else:
    ref_Count['n_train'] = (ref_Count['count'] * TRAIN_DIM) / 100
    ref_Count = ref_Count.round({'n_train' : 0})
    
    dt_ref['RN'] = dt_ref.sort_values(
        [TBL_SCHEMA['REF_COL']], ascending=[True]
    ).groupby([TBL_SCHEMA['REF_COL']]).cumcount() + 1
    
    for idx, row in ref_Count.iterrows():
        if idx == 0:
            dt_ref['is_train'] = numpy.where(
                (dt_ref['is_fire'] == row.is_fire) & (dt_ref.RN <= row.n_train),
                1, 0
            )
        else:
            dt_ref['is_train'] = numpy.where(
                (dt_ref['is_fire'] == row.is_fire) & (dt_ref.RN <= row.n_train),
                1, dt_ref.is_train
            )
    
    dt_cls   = dt_ref[dt_ref.is_train==0]
    dt_train = dt_ref[dt_ref.is_train==1]

In [ ]:
mdls = {
    'NB' : 'NaiveBayes', 'SVM' :'LinearSupportVectorMachine',
    'RF' : 'RandomForest', 'LOGREG':'LogisticRegression'}
"""
corr_words = correlated_words(
    dt_train, TBL_SCHEMA['REF_COL'], 'txtsan',
    os.path.join(OUT_FLD, 'corr_words_{}.xlsx'.format(str(TRAIN_DIM))),
    lang='english'
)"""

for m in mdls:
    tbl_cls = text_prediction(
        dt_train, dt_cls, TBL_SCHEMA['REF_COL'], 'txtsan', 'txtsan',
        os.path.join(OUT_FLD, 'res_{}_{}.xlsx'.format(m, str(TRAIN_DIM))),
        lang='english', method=mdls[m]
    )
    
    mtx = binary_eval(
        tbl_cls, TBL_SCHEMA['SAMPLE_FK'], TBL_SCHEMA['REF_COL'],
        tbl_cls, TBL_SCHEMA['SAMPLE_FK'],
        os.path.join(OUT_FLD, 'mtx_{}_{}.xlsx'.format(m, str(TRAIN_DIM))),
        tstCol = 'classification'
    )

In [ ]:
from glass.adv.txtcls.cls import get_rows_related_with_event
from glass.it import db_to_tbl

QT = (
    "(SELECT {t}.fid, {t}.{t_id}, {txtcol} AS txtcls, "
    "{dt}.{timecol} AS datahora, {t}.{refcol} "   
    "FROM {t} INNER JOIN {dt} ON "
    "{t}.{t_id} = {dt}.{dt_id}) AS foo"
).format(
    txtcol=TBL_SCHEMA['TXT_COL'], t=TBL_SCHEMA['SAMPLE_T'],
    dt=TBL_SCHEMA['DATA_T'], t_id=TBL_SCHEMA['SAMPLE_FK'],
    dt_id=TBL_SCHEMA['DATA_ID'], timecol=TBL_SCHEMA['TIME_COL'],
    refcol=TBL_SCHEMA['REF_COL']
)

tbl_ref = db_to_tbl(
    conDB, "SELECT * FROM {}".format(QT),
    os.path.join(OUT_FLD, 'tbl_ref.xlsx')
)

tbl_res = get_rows_related_with_event(conDB, {
    "TNAME" : QT,
    "TEXTCOL" : "txtcls",
    "TIMECOL" : "datahora",
    "SELCOL"  : ['fid', TBL_SCHEMA['SAMPLE_FK'], "datahora", TBL_SCHEMA['REF_COL']]
}, ['fog', 'incendi'], os.path.join(OUT_FLD, 'res_q.xlsx'))

mtx_q = binary_eval(
    tbl_ref, TBL_SCHEMA['SAMPLE_FK'], TBL_SCHEMA['REF_COL'],
    tbl_res, TBL_SCHEMA['SAMPLE_FK'],
    os.path.join(OUT_FLD, 'mtx_q.xlsx'),
    tstCol = None
)